In [51]:
import xlwings as xw
import pandas as pd
from ortools.linear_solver import pywraplp
import numpy as np

In [44]:
# settings
## to do: YAML these

sample_size = 100 # initial sample to optimize on

In [2]:
# wb = xw.Book()  # this will create a new workbook
wb = xw.Book(r'myproject/myproject.xlsm')  # connect to an existing file in the current working directory
# wb = xw.Book(r'C:\path\to\file.xlsx')  # on Windows: use raw strings to escape backslashes

Instantiate a sheet object.

In [4]:
sht = wb.sheets['temp_sheet']


Reading and writing is easy:

In [5]:
sht.range('A1').value = 'Foo 1'
sht.range('A1').value

'Foo 1'

Read in product and channel information from Excel.

In [39]:
customer_sheet = wb.sheets['customer_data']
product_sheet = wb.sheets['products']
channel_sheet = wb.sheets['channels']
scenario_sheet = wb.sheets['Scenario']

In [48]:
product_probs_all = customer_sheet.range('A1').options(pd.DataFrame, expand='table').value
products_df = product_sheet.range('A1').options(pd.DataFrame, expand='table').value

products = products_df.index
productValue = products_df.iloc[:,0]

channels_df = channel_sheet.range('A1').options(pd.DataFrame, expand='table').value
channels = channels_df.index
cost = channels_df['cost']
factor = channels_df['factor']

Get the available marketing budget from the `Scenario` sheet.

In [47]:
budget_range = scenario_sheet.range('budgetConstraints').value
availableBudget_total = budget_range[1]
availableBudget = availableBudget_total/sample_size # scale to sample_size for initial optimization

499.99949999999995

Create a sample of size `sample_size` for the initial optimization.

In [49]:
product_probs = product_probs_all.sample(n=sample_size, random_state=2058)

(100, 5)

Instantiate the solver as an MIP problem.

In [55]:
solver = pywraplp.Solver('SolveCampaignProblem', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

Define the number of customers, the number of offers and the number of channels as $x_{ijk}$.

In [56]:
num_customers = product_probs.shape[0]
num_products = len(products)
num_channels = len(channels)

x = {}

for i in range(num_customers):
    for j in range(num_products):
        for k in range(num_channels):
            x[i, j, k] = solver.IntVar(0, 1, 'x[%i,%i,%i]' % (i, j, k))

## Set up the constraints

  1. Offer only one product per customer. _(TO DO: update this.)_
  2. Adhere to budget, channel and product constraints from the Excel spreadsheet.
  3. Adhere to number of offer constraints
  


In [57]:
    ## offer only one product per customer
    for i in range(num_customers):
        solver.Add(solver.Sum([x[i, j, k] 
                               for j in range(num_products)
                               for k in range(num_channels)
                              ]) <= 1) # *** MAGIC NUMBER ALERT!!! ***

    ## Do not exceed the budget
    solver.Add(solver.Sum([x[i, j, k]*cost[k]
                           for i in range(num_customers)
                           for j in range(num_products)
                           for k in range(num_channels)
                          ]) <= availableBudget)

<ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x119637150> >

### Get the channel constraints

In [65]:
# channel minima
channelConstraints_n_range = scenario_sheet.range('channelConstraints_n').options(numbers=int).value
channelConstraints_df = pd.DataFrame(channelConstraints_n_range, index=channels, columns=['n_min', 'n_max'])

,n_min,n_max
channel,,
gift,1000,–
newsletter,1000,–
seminar,1000,–


### Set the channel constraints

In [68]:
# minimums for channel
for k in range(num_channels):
    solver.Add(solver.Sum([x[i, j, k]
        for i in range(num_customers)
        for j in range(num_products)
        ]) >= channelConstraints_df.loc[channels[k], 'n_min'])

There are many **convenience functions** available.

In [12]:
# read_value = sht.range("D5").options(numbers=int).value
read_value = sht.range("D4").options(numbers=int).value
print(read_value*2)
sht.range("D6").value = read_value * 2

600000


Read a named range called `test_value`.

In [13]:
test_value = sht.range("test_value").options(numbers=int).value
print(test_value)

500200


Write to the named range.

In [14]:
sht.range("test_value").value = test_value + 1
sht.range("test_value").value

500201.0

In [ ]:
sht.range('A1').value = [['Foo 1', 'Foo 2', 'Foo 3'], [10.0, 20.0, 30.0]]
sht.range('A1').expand().value

**Powerful converters** handle most data types of interest, including Numpy arrays and Pandas DataFrames in both directions:

In [17]:
import pandas as pd
df = pd.DataFrame([[1,2], [3,4]], columns=['a', 'b'])
sht.range('A1').value = df
sht.range('A1').options(pd.DataFrame, expand='table').value

,a,b
0.0,1.0,2.0
1.0,3.0,4.0


**Matplotlib figures** can be shown as pictures in Excel:

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure()
plt.plot([1, 2, 3, 4, 5])
sht.pictures.add(fig, name='MyPlot', update=True)

Shortcut for the active sheet: `xw.Range`

If you want to quickly talk to the active sheet in the active workbook, you don’t need instantiate a workbook and sheet object, but can simply do:

In [ ]:
xw.Range('A1').value = 'Foo'
xw.Range('A1').value
'Foo'

## 2. Macros: Call Python from Excel

